<a href="https://colab.research.google.com/github/SourLemon23/Eye-Net/blob/main/Cropping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
try:
    %tensorflow_version 2.x
except:
    pass

print('Using TensorFlow Version:', tf.__version__)

import tensorflow.keras.backend as K
from tensorflow import GradientTape

from tensorflow.image import resize
from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve, average_precision_score, PrecisionRecallDisplay

import os
from pathlib import Path
import csv
import nibabel as nib
import datetime

import numpy as np
import matplotlib.pyplot as plt
import cv2

! pip install -U efficientnet
from efficientnet.tfkeras import EfficientNetB7

Using TensorFlow Version: 2.5.0
     |████████████████████████████████| 51kB 4.0MB/s 


In [ ]:
# Authorize access to mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def load_data(dir):
    img_paths = []

    for root, dirs, files in os.walk(dir):
        for filename in files:
            abs_path = os.path.abspath(os.path.join(root, filename))
            img_paths.append(abs_path)

    return img_paths

In [ ]:
def process_data(file_paths, plot=False):
    processed_imgs = []

    for img_path in file_paths:
      img = cv2.imread(img_path)
      cropped_img = crop_to_eye(img, plot)
      processed_imgs.append(cropped_img)

    return processed_imgs

In [ ]:
def write_imgs(img_paths, processed_imgs, new_dir):
    for i, img_path in enumerate(img_paths):
        path_after_dir = r'/'.join(Path(img_path).parts[8:])
        final_img_path = Path(os.path.join(new_dir, path_after_dir))
        final_img_dir = final_img_path.parent

        # print('path_after_dir', path_after_dir)
        # print('final_img_path', final_img_path)
        # print('final_img_dir', final_img_dir)
        # print('---------')

        # This shouldn't ever run since we are not making any subfolders
        if os.path.isdir(str(final_img_dir)) == False:
            os.makedirs(final_img_dir)
            print('New directory made:', final_img_dir)

        cv2.imwrite(str(final_img_path), processed_imgs[i])

In [ ]:
def crop_to_eye(image, plot=False):
    img = image
    img2 = image.copy()

    grayscale_img = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    binary_img = cv2.threshold(grayscale_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

    contours, hierarchy = cv2.findContours(binary_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    areas = [cv2.contourArea(contour) for contour in contours]

    max_index = np.argmax(areas)
    contour = contours[max_index]
    cv2.drawContours(img2, contours, -1, (0, 255, 0), 3)

    x, y, w, h = cv2.boundingRect(contour)
    xmin = x
    xmax = x + w
    ymin = y
    ymax = y + h
    cv2.rectangle(img2, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)

    cropped_img = img[ymin:ymax, xmin:xmax]

    if plot:
        fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(24, 4), constrained_layout=True)
        
        for i, img_filter in enumerate(['Grayscale', 'Binary', 'Contoured', 'Cropped']):
            axs[i].set_title(img_filter)

        axs[0].imshow(grayscale_img)
        axs[1].imshow(binary_img)
        axs[2].imshow(img2)
        axs[3].imshow(cropped_img)

        plt.show()

    return cropped_img

In [ ]:
# Crop images within a directory
dir = r'/content/drive/MyDrive/Colab Notebooks/Diabetic Retinopathy/APTOS_Dataset/Images'
new_dir = r'/content/drive/MyDrive/Colab Notebooks/Diabetic Retinopathy/cropped_APTOS_Dataset'

img_paths = load_data(dir)[3000:]
print('# img paths:', len(img_paths))
print(img_paths)

# img paths: 418
['/content/drive/MyDrive/Colab Notebooks/Diabetic Retinopathy/APTOS_Dataset/Images/687759336b0d.png', '/content/drive/MyDrive/Colab Notebooks/Diabetic Retinopathy/APTOS_Dataset/Images/686ed1dbae20.png', '/content/drive/MyDrive/Colab Notebooks/Diabetic Retinopathy/APTOS_Dataset/Images/6a2c3f4ef329.png', '/content/drive/MyDrive/Colab Notebooks/Diabetic Retinopathy/APTOS_Dataset/Images/69f43381317b.png', '/content/drive/MyDrive/Colab Notebooks/Diabetic Retinopathy/APTOS_Dataset/Images/697538183db5.png', '/content/drive/MyDrive/Colab Notebooks/Diabetic Retinopathy/APTOS_Dataset/Images/69df7ade0575.png', '/content/drive/MyDrive/Colab Notebooks/Diabetic Retinopathy/APTOS_Dataset/Images/6b00cb764237.png', '/content/drive/MyDrive/Colab Notebooks/Diabetic Retinopathy/APTOS_Dataset/Images/6b30767595d8.png', '/content/drive/MyDrive/Colab Notebooks/Diabetic Retinopathy/APTOS_Dataset/Images/6b7cf869622a.png', '/content/drive/MyDrive/Colab Notebooks/Diabetic Retinopathy/APTOS_Datase

In [ ]:
processed_imgs = process_data(img_paths, plot=False)
print('# imgs processed:', len(processed_imgs))

# imgs processed: 418


In [ ]:
write_imgs(img_paths, processed_imgs, new_dir)